## Check List 4 - PlanningPipeline
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4.1 PlanningScene**  
  - create_binder
    - actor: PlacePlane, Gripper2Tool, SweepTool
    - handle: Grasp2Point, PlacePoint, SweepPoint
  - create_subject
    - subject: CustomObject, SweepLineTask
  - initialize_state
* **4.2 MotionPlanner**  
  - MoveitPlanner
    - pick, place, sweep
    - Interpolated path generation by incremental IK
* **4.3 TaskPlanner**  
  - RRT
    - pick & place & sweep
      - single process
      - multi process
  - PDDLStream
    - pick & place

## set running directory to project source

In [1]:
import os
import sys
import numpy as np
import time
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7kiro, ((0,0,0), (0,0,0)), 
                "192.168.21.6")]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
==== Kiro Tool connected ====
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


In [3]:
from demo_utils.environment import add_brush

In [4]:

add_brush(gscene, face_name="brush_fase", tool_link="indy0_tcp",
          thickness=0.05, tool_dim=(0.15,0.32))

##### add geometries

In [5]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=np.random.rand(3)*0, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

##### create PlanningScene

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

In [7]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene)

In [9]:
brush_face = gscene.NAME_DICT["brush_fase"]

In [11]:
brush_face.draw_traj_coords([crob.home_pose])

[array([[ 9.39692577e-01, -6.05567895e-12,  3.42020150e-01,
          1.40910375e-01],
        [ 8.06369950e-12,  1.00000000e+00, -4.44921114e-12,
         -1.86499992e-01],
        [-3.42020126e-01,  6.93883854e-12,  9.39692568e-01,
          4.74998740e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]])]

In [60]:
T1 = SE3(np.identity(3), (0.55,0, 0.1))

In [61]:
gscene.add_highlight_axis("hl", "t1", T=T1)

In [62]:
HOME_POSE = [0, 0, -np.pi / 2, 0, -np.pi / 4, 0, -np.pi/4]

In [63]:
mplan.validate_trajectory([HOME_POSE])

True

In [64]:
gscene.show_pose(HOME_POSE)

In [65]:
traj, succ = mplan.get_incremental_traj("indy0_tcp", brush_face.Toff, T1,
                           HOME_POSE, step_size=0.01, ERROR_CUT=0.01,
                           SINGULARITY_CUT=0.01, VERBOSE=True,
                           ref_link="base_link", VISUALIZE=False)

end


In [66]:
gscene.show_motion(traj)

In [67]:
Qlast = traj[-1]

In [68]:
T2 = SE3(np.identity(3), (0.3,0, 0.1))

In [69]:
traj, succ = mplan.get_incremental_traj("indy0_tcp", brush_face.Toff, T2,
                           Qlast, step_size=0.01, ERROR_CUT=0.01,
                           SINGULARITY_CUT=0.01, VERBOSE=True,
                           ref_link="base_link", VISUALIZE=False)

end


In [70]:
gscene.show_motion(traj)

In [32]:
crob.reset_connection(True)

connection command:
indy0: True
==== Kiro Tool connected ====


In [33]:
indy = crob.robot_dict["indy0"]

In [34]:
crob.simulator.set_gscene(gscene)

In [72]:
Qcur = crob.get_real_robot_pose()

In [73]:
gscene.show_pose(Qcur)

In [74]:
diff = Qlast - Qcur

In [75]:
crob.joint_move_make_sure(Qcur+diff*1.0)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


In [123]:

traj_new = bspline_wps(dt_step=2e-2, trajectory_simp=traj,
                       vel_lims=0.05, acc_lims=0.05, radii_deg=1)

In [124]:
traj_rev = np.array(list(reversed(traj_new)))

In [125]:
len(traj_new)

430

In [114]:
crob.move_joint_traj(traj_new)

Connect: Server IP (192.168.21.6)


True

In [81]:
gscene.show_motion(traj_rev)

In [115]:
crob.move_joint_traj(traj_rev)

Connect: Server IP (192.168.21.6)


True

In [83]:

import requests
from bs4 import BeautifulSoup
from time import sleep

def switch_command(ip_addr, on_off, UI_PORT=9990):
    uri = "http://{ip_addr}:{UI_PORT}/param_setting?switch_control0={on_off}".format(ip_addr=ip_addr, UI_PORT=UI_PORT, on_off=int(on_off))
    print(uri)
    requests.get(uri)

In [85]:
INDY_IP = "192.168.21.6"

In [86]:

uri = "http://{ip_addr}:{UI_PORT}/param_setting?switch_frame0={on_off}".format(
    ip_addr=INDY_IP, UI_PORT=9990, on_off=1)
requests.get(uri)

<Response [200]>

In [126]:
crob.joint_move_make_sure(traj_new[0])

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


In [127]:
switch_command(INDY_IP, True)

http://192.168.21.6:9990/param_setting?switch_control0=1


In [128]:
crob.move_joint_traj(traj_new)

Connect: Server IP (192.168.21.6)


True

In [129]:
crob.move_joint_traj(traj_rev)

Connect: Server IP (192.168.21.6)


True

In [130]:
switch_command(INDY_IP, False)

http://192.168.21.6:9990/param_setting?switch_control0=0


In [131]:
crob.joint_move_make_sure(traj_rev[-1])

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
